import tweepy
import json

In [160]:
import pyodbc
from __future__ import absolute_import, print_function
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from sqlalchemy.exc import ProgrammingError
import tweepy
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import math, re, string, requests, json



In [161]:
conn = pyodbc.connect(r'DSN=starbucks')
curr= conn.cursor()
analyzer = SentimentIntensityAnalyzer()


language_codes = ["fr", "de", "es", "it", "ru", "ja", "ar", "zh"]


apikey = 'lQr03iVJ9Rs3kVBOlZt07ak7q'
apisecret ='mght8IbXaNJ2au57BabPZAqIFh4USJJF4V5GTXGPKdbVCdhd3u'
accesstoken = '887695854076915712-bIE18SRELRjCFp5x0iq469G2vb02HRw'
tokensecret = 'jEJREL1NQhrI0galiaGqDIf9hu6D5mEo5Hjj1suH45YIn'


In [175]:
class StdOutListener(StreamListener):
    def on_status(self, status):
    
        if status.user.lang in language_codes:
            api_url = "http://mymemory.translated.net/api/get?q={}&langpair={}|{}".format(status.text, status.user.lang, "en")
            hdrs ={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
            response = requests.get(api_url, headers=hdrs)
            response_json = json.loads(response.text)
            translation = response_json["responseData"]["translatedText"]
            translator_name = "MemoryNet Translation Service"
            vs = analyzer.polarity_scores(translation)
            sent = vs['compound']
            print(non-eng)
        else:
            vs = analyzer.polarity_scores(status.text)
            sent = vs['compound']
            print(eng)
        
        status.text = str.replace(status.text, '\'', '\'\'')
        q= 'insert into starbucks(\
        id_str, created,Uk_time, text, fav,name, followers, sentiment, language)\
        values (\'%s\',\'%s\', getdate(),\'%s\',\'%s\',\'%s\',\'%s\', \'%s\',\'%s\,\'%s\')'\
        %(status.id_str, status.created_at, status.text, status.favorite_count, status.user.screen_name,\
          status.user.followers_count, sent, status.user.lang, translation)
        
     
        try:
            curr.execute(q)
            conn.commit()
            
        except:
            print(q)
        #('insert into starbucks(\
        #id_str, created,text, fav,name, description, loc, user_created, follower)\
        #values (%s,%s,%s,%s,%s,%s,%s,%s,%s)'\
        #%(status.id_str, status.created_at, status.text, status.favorite_count, status.user.screen_name, status.user.description, status.user.location, status.user.created_at, status.user.followers_count))
        
        
         
               
    def on_error(self, status_code):
        if status_code == 420:
            return False



In [196]:
l = StdOutListener()
auth = tweepy.OAuthHandler(apikey, apisecret)
auth.set_access_token(accesstoken, tokensecret)
api = tweepy.API(auth)
stream = Stream(auth, l)
stream.filter(track=['starbucks'])